# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f97d0ce2f70>
├── 'a' --> tensor([[ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155]])
└── 'x' --> <FastTreeValue 0x7f97d0ce2f40>
    └── 'c' --> tensor([[-0.7046,  1.8036, -0.8145,  0.0660],
                        [-0.6666,  0.7362, -1.1966, -1.5355],
                        [ 0.4917,  0.7246,  1.7775,  0.4924]])

In [4]:
t.a

tensor([[ 0.5932, -0.7008, -0.7331],
        [-1.7363,  0.5790,  0.1155]])

In [5]:
%timeit t.a

58.9 ns ± 0.0922 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f97d0ce2f70>
├── 'a' --> tensor([[ 1.5800, -0.4505, -0.3070],
│                   [ 0.7781, -0.3092, -1.1490]])
└── 'x' --> <FastTreeValue 0x7f97d0ce2f40>
    └── 'c' --> tensor([[-0.7046,  1.8036, -0.8145,  0.0660],
                        [-0.6666,  0.7362, -1.1966, -1.5355],
                        [ 0.4917,  0.7246,  1.7775,  0.4924]])

In [7]:
%timeit t.a = new_value

65.9 ns ± 0.0607 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.5932, -0.7008, -0.7331],
               [-1.7363,  0.5790,  0.1155]]),
    x: Batch(
           c: tensor([[-0.7046,  1.8036, -0.8145,  0.0660],
                      [-0.6666,  0.7362, -1.1966, -1.5355],
                      [ 0.4917,  0.7246,  1.7775,  0.4924]]),
       ),
)

In [10]:
b.a

tensor([[ 0.5932, -0.7008, -0.7331],
        [-1.7363,  0.5790,  0.1155]])

In [11]:
%timeit b.a

59.6 ns ± 0.0494 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2744, -0.3955, -1.1192],
               [-0.6843, -0.8284,  0.6274]]),
    x: Batch(
           c: tensor([[-0.7046,  1.8036, -0.8145,  0.0660],
                      [-0.6666,  0.7362, -1.1966, -1.5355],
                      [ 0.4917,  0.7246,  1.7775,  0.4924]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.158 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

886 ns ± 5.37 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.5 µs ± 46.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 393 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 293 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f97ac187b80>
├── 'a' --> tensor([[[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]],
│           
│                   [[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]],
│           
│                   [[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]],
│           
│                   [[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]],
│           
│                   [[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]],
│           
│                   [[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]],
│           
│                   [[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]],
│           
│                   [[ 0.5932, -0.7008, -0.7331],
│                    [-1.7363,  0.5790,  0.1155]]])
└── 'x' --> <FastTreeValue 0x7f97ac187df0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.1 µs ± 67.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9723908820>
├── 'a' --> tensor([[ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155],
│                   [ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155],
│                   [ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155],
│                   [ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155],
│                   [ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155],
│                   [ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155],
│                   [ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155],
│                   [ 0.5932, -0.7008, -0.7331],
│                   [-1.7363,  0.5790,  0.1155]])
└── 'x' --> <FastTreeValue 0x7f97ac191f40>
    └── 'c' --> tensor([[-0.7046,  1.8036, -0.8145,  0.0660],
                        [-0.6666,  0.7362, -1.1966, -1.5355],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 69 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.8 µs ± 143 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.7046,  1.8036, -0.8145,  0.0660],
                       [-0.6666,  0.7362, -1.1966, -1.5355],
                       [ 0.4917,  0.7246,  1.7775,  0.4924]],
              
                      [[-0.7046,  1.8036, -0.8145,  0.0660],
                       [-0.6666,  0.7362, -1.1966, -1.5355],
                       [ 0.4917,  0.7246,  1.7775,  0.4924]],
              
                      [[-0.7046,  1.8036, -0.8145,  0.0660],
                       [-0.6666,  0.7362, -1.1966, -1.5355],
                       [ 0.4917,  0.7246,  1.7775,  0.4924]],
              
                      [[-0.7046,  1.8036, -0.8145,  0.0660],
                       [-0.6666,  0.7362, -1.1966, -1.5355],
                       [ 0.4917,  0.7246,  1.7775,  0.4924]],
              
                      [[-0.7046,  1.8036, -0.8145,  0.0660],
                       [-0.6666,  0.7362, -1.1966, -1.5355],
                       [ 0.4917,  0.7246,  1.7775,  0.4924]],

In [26]:
%timeit Batch.stack(batches)

78 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.7046,  1.8036, -0.8145,  0.0660],
                      [-0.6666,  0.7362, -1.1966, -1.5355],
                      [ 0.4917,  0.7246,  1.7775,  0.4924],
                      [-0.7046,  1.8036, -0.8145,  0.0660],
                      [-0.6666,  0.7362, -1.1966, -1.5355],
                      [ 0.4917,  0.7246,  1.7775,  0.4924],
                      [-0.7046,  1.8036, -0.8145,  0.0660],
                      [-0.6666,  0.7362, -1.1966, -1.5355],
                      [ 0.4917,  0.7246,  1.7775,  0.4924],
                      [-0.7046,  1.8036, -0.8145,  0.0660],
                      [-0.6666,  0.7362, -1.1966, -1.5355],
                      [ 0.4917,  0.7246,  1.7775,  0.4924],
                      [-0.7046,  1.8036, -0.8145,  0.0660],
                      [-0.6666,  0.7362, -1.1966, -1.5355],
                      [ 0.4917,  0.7246,  1.7775,  0.4924],
                      [-0.7046,  1.8036, -0.8145,  0.0660],
                   

In [28]:
%timeit Batch.cat(batches)

144 µs ± 292 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

305 µs ± 2.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
